<ul class="breadcrumb">
  <li><a href="2.1.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="2.3.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font><br> 
  <font size="5" face="Georgia">(with Maternal Effects)</font> 

</div>

In [1]:
using DataFrames,JWAS,JWAS.Datasets

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multivariate Linear Additive Genetic Model</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [28]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [29]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### pedigree

In [30]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [31]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [32]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [33]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [34]:
model    = build_model(model_equations,R);

In [35]:
set_covariate(model,"age");

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [36]:
ped = get_pedigree(pedfile);

Finished!


In [37]:
set_random(model,"Animal", ped,G)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [42]:
out = runMCMC(model,data,chain_length=1000,printout_frequency=500,output_samples_frequency=10);

MCMC Information:
methods                        conventional (no markers)
chain_length                                   1000
estimatePi                                    false
constraint                                    false
missing_phenotypes                            false
starting_value                                false
output_samples_frequency                         10
printout_frequency                              500

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000




Posterior means at iteration: 500
Residual covariance matrix: 
[12.000351 2.623073
 2.623073 0.783612]
Polygenic effects covariance matrix 
[528.873557 140.488269
 140.488269 40.09335]

running MCMC for conventional (no markers)... 57%|███   |  ETA: 0:00:00
Posterior means at iteration: 1000
Residual covari

In [43]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 5 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Posterior mean of residual covariance matrix"
  "MCMC samples for residual covariance matrix"
  "MCMC samples for polygenic effects covariance matrix"
  "Posterior mean of location parameters"

In [44]:
out2["MCMC samples for residual covariance matrix"]

4x100 Array{Float64,2}:
 149.849    93.3118   58.9306  49.4871  …  2.78258  4.85405   2.82043 
  23.5626   17.5878   18.8604  22.0215     1.54568  0.686716  0.581066
  23.5626   17.5878   18.8604  22.0215     1.54568  0.686716  0.581066
   3.95223   3.57914   6.5345  10.1299     1.67531  0.499512  0.329622

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. The Multivariate Linear Additive Genetic Model with Maternal Effects</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [70]:
phenofile = Datasets.dataset("testMT","maternal.txt")
data=readtable(phenofile)

,Animal,BW,CW,age,sex,dam
1,S1,100.0,10.0,8,M,0
2,D1,50.0,12.9,7,F,0
3,O1,150.0,13.0,3,M,D1
4,O3,40.0,5.0,4,F,D1


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [71]:
model_equations = "BW = intercept + age + sex + Animal+ dam;
                   CW = intercept + age + sex + Animal";

In [72]:
model = build_model(model_equations,R);
set_covariate(model,"age");

In [73]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
set_random(model,"Animal dam", ped,G0)

In [74]:
out=runMCMC(model,data,chain_length=1000,printout_frequency=500);

MCMC Information:
methods                        conventional (no markers)
chain_length                                   1000
estimatePi                                    false
constraint                                    false
missing_phenotypes                            false
starting_value                                false
output_samples_frequency                          0
printout_frequency                              500

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000



running MCMC for conventional (no markers)... 39%|██    |  ETA: 0:00:00
Posterior means at iteration: 500
Residual covariance matrix: 
[132.284208 32.582568
 32.582568 8.604643]
Polygenic effects covariance matrix 
[8.242737 1.591939 1.006508
 1.591939 1.041052 0.088355
 1.006508 0.088355 0.762292]

running 

In [75]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 5 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Posterior mean of residual covariance matrix"
  "MCMC samples for residual covariance matrix"
  "MCMC samples for polygenic effects covariance matrix"
  "Posterior mean of location parameters"